In [ ]:
# default_exp play01


In [ ]:
#export
import time
import pathlib
import numpy as np
import cv2 as cv
import pandas as pd


In [ ]:
#export
_code_git_version="995b5c34dc5a9e719324ff6a72f86ed15dc0f8f9"
_code_repository="https://github.com/plops/cl-py-generator/tree/master//example/76_opencv_cuda/source/01_play.ipynb"
_code_generation_time="18:16:41 of Wednesday, 2024-04-24 (GMT+1)"
start_time=time.time()
debug=True


In [ ]:
#export
df_status=pd.DataFrame([dict(name="numpy", version=np.__version__), dict(name="cv2", version=cv.__version__), dict(name="pandas", version=pd.__version__)])
print(df_status)


In [ ]:
#export
# https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
fns=list(pathlib.Path("data/").glob("left*.jpg"))
print(fns)


In [ ]:
#export
criteria=(((cv.TERM_CRITERIA_EPS)+(cv.TERM_CRITERIA_MAX_ITER)),30,(1.00e-3),)
objp=np.zeros([((6)*(7)), 3], np.float32)
objp[:,:2]=np.mgrid[0:7,0:6].T.reshape(-1, 2)
objpoints=[]
imgpoints=[]
for fn in fns:
    img=cv.imread(str(fn))
    gray=cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, corners=cv.findChessboardCorners(gray, [7, 6], None)
    if ( ret ):
        objpoints.append(objp)
        corners2=cv.cornerSubPix(gray, corners, [11, 11], [-1, -1], criteria)
        imgpoints.append(corners)
        cv.drawChessboardCorners(img, [7, 6], corners2, ret)
        cv.imshow("img", img)
        cv.waitKey(500)
cv.destroyAllWindows()


In [ ]:
#export
ret, mtx, dist, rvecs, tvecs=cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(mtx)


In [ ]:
#export
img=cv.imread(str(fns[0]))
h, w=img.shape[:2]
new_mtx, roi=cv.getOptimalNewCameraMatrix(mtx, dist, [w, h], 1, [w, h])
print(new_mtx)


In [ ]:
#export
dst=cv.undistort(img, mtx, dist, None, new_mtx)
x, y, w, h=roi
dst=dst[y:((y)+(h)),x:((x)+(w))]
cv.imshow("dst", dst)
cv.waitKey(5000)
cv.destroyAllWindows()


In [ ]:
#export
res=(0.    )
for i in range(len(objpoints)):
    imgpoints2, _=cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    err=((cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2))/(len(imgpoints2)))
    res += err
print("mean reprojection error: {:5.3f}px".format(((res)/(len(objpoints)))))
